<h1>One Hot Encoding und Dummy-Variablen</h1>

Wenn ein Dataset Text enthält, egal ob der Text eine numerische Bedeutung hat wie "Zwei", muss dieser in eine numerische Repräsentation umgeformt werden, damit die Modelle damit umgehen könne. 


<i>Abb1:</i> Übersicht der Datentypen 
<center>
    <img src="./files_data/img/DataTypes_encode_Notebook_1.PNG" width=800 hight=1000 >
</center>

Bei einem gegebenen Dataset mit verschiedenen Stadtnamen, wird OnHotEncode angewendet. 

<i>Abb2:</i> On Hot Encode


<img src="./files_data/img/DataTypes_encode_Notebook_2.PNG" width=600 hight=5000 >


<i>Abb3:</i> Integer / Label Encoding


<img src="./files_data/img/DataTypes_encode_Notebook_3.PNG" width=500 hight=400 >

On Hot Encoding: weise binäre Werte zu wie 0 und 1 oder True und False. <br>Für jede Kategorie eine Spalte.

Die erstellten Variablen nennt man auch Dummy-Variablen

Label Encoding: weise fortlaufende Zahlen zu, die je eine Stadt repräsentiert. <br>Eine Spalte für jede Kategorie

Die nominalen Daten folgen keiner bestimmten Reihenfolge und haben keine Relation zueinander. Daher wird hier eher On Hot Encoding / Label Encoding genutzt.

Bei den anderen, wo es eine Reihenfolge gibt, wie zum Beispiel "Bachelor, Master, ...", wird ein Ordinal Encoding benötigt. 

Bei dem Label Encoding kann eine Relation zwischen den Daten hergestellt werden (Köln + Alsfeld = Fulda oder Köln > Fulda). Diese Annahmen können zu falschen Schlussfolgerungen führen.

Sklearn und Pandas bieten gute Möglichkeiten, die betroffenen Spalten umzuformen. 

Beispiel Dataset: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction [Letzter Zugriff 04.06.2024] 

> fedesoriano. (September 2021). Heart Failure Prediction Dataset. Retrieved [Date Retrieved]<br> from https://www.kaggle.com/fedesoriano/heart-failure-prediction.

Dieses Dataset kann gut für die Veranschaulichung genutzt werden. 

In [35]:
import pandas as pd
import numpy  as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [14]:
heart_data = pd.read_csv("./files_data/data/heart_failure_prediction.zip", compression='zip')
heart_data.head(2)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1


In [15]:
heart_data2 = heart_data[ ['Sex', 'ChestPainType', 'ST_Slope' ] ] # 'ChestPainType', 'ST_Slope'
heart_data2.head(3)

,Sex,ChestPainType,ST_Slope
0,M,ATA,Up
1,F,NAP,Flat
2,M,ATA,Up


Mit nur einem Aufruf erstellt Pandas die Dummy-Variablen. Der Datentyp kann von Boolean zu Integer umgestellt werden.

In [16]:
# Gibt die Tabelle mit den Dummies zurück.
# - On Hot Encode - #
pd.get_dummies(heart_data['Sex']) # dtype="int" => Int statt Bool.

,F,M
0,False,True
1,True,False
2,False,True
3,True,False
4,False,True
...,...,...
913,False,True
914,False,True
915,False,True
916,True,False


Diese Tabelle kann vielseitig genutzt werden. Um es für das Training zu nutzen, wird es an das Dataframe angeknüpft, welches für das Training genutzt wird.

In [17]:
# Erstelle Dummies, dann füge die Dataframes zusammen. 
sex_column_dummy = pd.get_dummies(heart_data['Sex'], dtype="int")
heart_data2 = pd.concat([heart_data2, sex_column_dummy], axis="columns")
heart_data2.head()

,Sex,ChestPainType,ST_Slope,F,M
0,M,ATA,Up,0,1
1,F,NAP,Flat,1,0
2,M,ATA,Up,0,1
3,F,ASY,Flat,1,0
4,M,NAP,Up,0,1


<i>Abb4:</i> 2 Spalten müssen weg.

<img src="./files_data/img/DataTypes_encode_Notebook_4.PNG" width=300 hight=300 >

Der nächste Schritt ist, die Spalte "Sex" zu löschen, da diese nicht mehr benötigt wird. 

Eine der Spalten "F" oder "M" muss ebenfalls entfernt werden, da es sonst Redundanz und Multilinearität erzeugt => Dummy Variable Trap. <br>

Wenn es viele Features gibt mit vielen unterschiedlichen Werten => curse of dimensionality <br>
Es beschreibt das Problem, dass durch dieses Encoding die Dimension des Datasets schnell sehr groß wird, was das Model Leistungstechnisch negativ beeinflusst.

In [18]:
# 2 Spalten werden gelöscht
heart_data3 = heart_data2.drop(['Sex', 'F'], axis="columns")
heart_data3.head(3)

,ChestPainType,ST_Slope,M
0,ATA,Up,1
1,NAP,Flat,0
2,ATA,Up,1


Die oberen Operationen kann Pandas auch direkt ausführen, wie die untere Zelle zeigt. 

In [99]:
# Parameter: #
# drop_first=True: lässt Spalte einer On Hot Encode Spaplte fallen. Kein manuelles Löschen mehr nötig. 
# dtype='int'    : Datentyp soll Int statt Bool.
tmp_df = heart_data[['Sex', 'ST_Slope', 'Age']]      # Erstelle kleineres df für bessere Übersicht. 
pd.get_dummies(tmp_df, columns=['Sex', 'ST_Slope'], drop_first=True, dtype='int')  # On Hot Encode auf "Sex" und "ST_Slope"

,Age,Sex_M,ST_Slope_Flat,ST_Slope_Up
0,40,1,0,1
1,49,0,1,0
2,37,1,0,1
3,48,0,1,0
4,54,1,0,1
...,...,...,...,...
913,45,1,1,0
914,68,1,1,0
915,57,1,1,0
916,57,0,1,0


Dasselbe ist auch mit Sklearn möglich. 

In [25]:
chestPainType = heart_data3['ChestPainType']  # Separat
chestPainType.unique()                        # 4 Werte 

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [34]:
# Integer / Label Encoder # 
label_enc = LabelEncoder()  # Encoder 

encoded_chestPainTypelabel   = label_enc.fit_transform(chestPainType)  # Erstelle Encoding

np.unique(encoded_chestPainTypelabel) # Ergebnis-  0 bis 3 für diese 4 Werte. Kein On Hot Encode.

array([0, 1, 2, 3])

Dann gibt es noch mehrere Wege On Hot Encode mit Sklearn umzusetzen.

In [60]:
from sklearn.compose import ColumnTransformer
heart_data4 = heart_data3  # copy

In [91]:
# On Hot Encode mit ColumnTransformer 
ohe    = OneHotEncoder(sparse_output=False)
ct     = ColumnTransformer([("ChestPainType", ohe, [0])], remainder = 'passthrough')
result = ct.fit_transform(heart_data3)
result

array([[0.0, 1.0, 0.0, 0.0, 'Up', 1],
       [0.0, 0.0, 1.0, 0.0, 'Flat', 0],
       [0.0, 1.0, 0.0, 0.0, 'Up', 1],
       ...,
       [1.0, 0.0, 0.0, 0.0, 'Flat', 1],
       [0.0, 1.0, 0.0, 0.0, 'Flat', 0],
       [0.0, 0.0, 1.0, 0.0, 'Up', 1]], dtype=object)

In [92]:
# On Hot Encode ohne ColumnTransformer
sklearn_ohe     = OneHotEncoder(sparse_output=False)   # sparse_output: Wenn True, ist die Ausgabe ein:  scipy.sparse.csr_matrix
ohe_encoded     = sklearn_ohe.fit_transform(heart_data4[ ['ChestPainType'] ])
ohe_encoded

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [93]:
columns_names_df = pd.DataFrame(ohe_encoded, columns=sklearn_ohe.get_feature_names_out(['ChestPainType']), dtype='int')
columns_names_df

,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,0,1,0,0
1,0,0,1,0
2,0,1,0,0
3,1,0,0,0
4,0,0,1,0
...,...,...,...,...
913,0,0,0,1
914,1,0,0,0
915,1,0,0,0
916,0,1,0,0


In [66]:
# Oder so:
categorical_data = heart_data4.select_dtypes(include=['object']).columns.tolist()
categorical_data # Zwei Spalten mit kategorischen Daten. 

['ChestPainType', 'ST_Slope']

In [70]:
sklearn_ohe     = OneHotEncoder(sparse_output=False)   # sparse_output: Wenn True, ist die Ausgabe ein:  scipy.sparse.csr_matrix
ohe_encoded     = sklearn_ohe.fit_transform(heart_data4[categorical_columns])
ohe_encoded

array([[0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.]])

Jetzt können noch die Featurenamen extrahiert und eingefügt werden. 

In [73]:
columns_names_df = pd.DataFrame(ohe_encoded, columns=sklearn_ohe.get_feature_names_out(categorical_data), dtype='int')
columns_names_df

,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,0,1,0,0,0,0,1
1,0,0,1,0,0,1,0
2,0,1,0,0,0,0,1
3,1,0,0,0,0,1,0
4,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...
913,0,0,0,1,0,1,0
914,1,0,0,0,0,1,0
915,1,0,0,0,0,1,0
916,0,1,0,0,0,1,0


Das Dataframe kann dann wieder an ein anderes Dataframe angehangen werden.

In [75]:
merged_df = pd.concat([heart_data4, columns_names_df], axis="columns")
merged_df.head()

,ChestPainType,ST_Slope,M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,ATA,Up,1,0,1,0,0,0,0,1
1,NAP,Flat,0,0,0,1,0,0,1,0
2,ATA,Up,1,0,1,0,0,0,0,1
3,ASY,Flat,0,1,0,0,0,0,1,0
4,NAP,Up,1,0,0,1,0,0,0,1


In [77]:
# Endergebnis
merged_df.drop(['ChestPainType', 'ST_Slope', 'ChestPainType_ASY', 'ST_Slope_Down'], axis="columns")

,M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,ST_Slope_Flat,ST_Slope_Up
0,1,1,0,0,0,1
1,0,0,1,0,1,0
2,1,1,0,0,0,1
3,0,0,0,0,1,0
4,1,0,1,0,0,1
...,...,...,...,...,...,...
913,1,0,0,1,1,0
914,1,0,0,0,1,0
915,1,0,0,0,1,0
916,0,1,0,0,1,0


In [ ]:
# // Content coming